# LIBRARIES

In [1]:
import os
import ast
import math
import ccxt
import time
import json
import pickle
import numpy as np
import configparser
import pandas as pd
import google.cloud
from pandas_gbq import gbq
import matplotlib.cm as cm
from boruta import BorutaPy
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
from google.cloud import bigquery
from sklearn import preprocessing
from requests import Request, Session
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix
from scipy.interpolate import UnivariateSpline
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

#_____

pd.set_option('display.max_rows', 5000)
pd.options.mode.chained_assignment = None

# FUNCTIONS

In [2]:
#FUNCIÓN QUE LEE EL ARCHIVO CONFIG
def get_config(category, key):
    
    global config
    return config[category][key]

#_____

#FUNCIÓN QUE GENERA CONEXIÓN CON ARCHIVO CONFIG
def updateConfig(config_name):
    
    global config
    
    config = configparser.ConfigParser()
    config.sections()
    config.read(config_name)
    
#_____

#FUNCIÓN QUE GRAFICA POR COLORES SEGUN COMPRAS Y NO COMRPAS
def plot_colourline(x,y,c):
    c = c
    ax = plt.gca()
    for i in np.arange(len(x)-1):
        ax.plot([x[i],x[i+1]], [y[i],y[i+1]], c=c[i])
    return

#_____

#FUNCIÓN QUE ESCRIBE PICKLE DE VARIABLE
def writePickleVariable(variable,variable_name):
    pickle_out = open(variable_name+".pickle","wb")
    pickle.dump(variable, pickle_out)
    pickle_out.close()
    
#_____
    
#FUNCIÓN QUE LEE PICKLE DE VARIABLE
def readPickleVariable(variable_name):    
    return pickle.load(open(variable_name+".pickle","rb"))

#_____

#FUNCIÓN QUE REDONDEA FLOAT HACIA ABAJO SEGÚN DECIMALES
def round_decimals_down(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

#_____

#FUNCIÓN QUE REDONDEA FLOAT HACIA ARRIBA SEGÚN DECIMALES
def round_decimals_up(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

#_____

#FUNCIÓN QUE DESCARGA BASE DE DATOS DE MERCADO DE BIGQUERY
def downloadDataBaseBigQuery(market_model,rows=None):
    
    #_____GOOGLE CLOUD CONECTION
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/bigQueryAccess.json"

    #_____SI NO SE DA UN NÚMERO ESPECÍFICO DE FILAS POR PARÁMETRO
    if rows==None:
    
        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC",project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
        
    else:
        
        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet."+market_model.replace("/","_")+"] ORDER BY TIME DESC LIMIT "+str(rows),project_id="dogwood-terra-308100",dialect="legacy").sort_values(by="TIME")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)

    candlesDataBase_BigQuery.OPEN=candlesDataBase_BigQuery.OPEN.astype(float)
    candlesDataBase_BigQuery.HIGH=candlesDataBase_BigQuery.HIGH.astype(float)
    candlesDataBase_BigQuery.LOW=candlesDataBase_BigQuery.LOW.astype(float)
    candlesDataBase_BigQuery.CLOSE=candlesDataBase_BigQuery.CLOSE.astype(float)
    candlesDataBase_BigQuery.VOLUME=candlesDataBase_BigQuery.VOLUME.astype(float)
    
    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE DEVUELVE EL RSI DADO UN PERIODO DETERMINADO
def RSI(candlesDataBase_BigQuery):
    
    global config
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    series=pd.Series(list(candlesDataBase_BigQuery.CLOSE.values))
    period=int(get_config("PARAMETERS","RSI_LEN"))
    
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    
    rsi=100 - 100 / (1 + rs)
    
    for i in list(rsi.index.values):
        candlesDataBase_BigQuery.at[i,"RSI"]=rsi[i]
                                        
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE CALCULA MACD
def MACD(candlesDataBase_BigQuery):
    
    global config
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    close_data_df=candlesDataBase_BigQuery.copy()
    close_data_df.CLOSE=close_data_df.CLOSE.astype(float)
    close_data_df=close_data_df[["CLOSE"]]
    
    #_____CREAR VARIABLE MACD
    macd_12 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_SHORT")), adjust=False).mean()
    macd_26 = close_data_df.ewm(span=int(get_config("PARAMETERS","MACD_LONG")), adjust=False).mean()
    macd = macd_12 - macd_26
    macd_9 = macd.ewm(span=int(get_config("PARAMETERS","MACD_TRIGGER")), adjust=False).mean()

    #_____AGREGAR VARIABLES A LA BASE DE DATOS
    candlesDataBase_BigQuery["MACD"]=macd
    candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_SHORT")]=macd_12
    candlesDataBase_BigQuery["MACD_"+get_config("PARAMETERS","MACD_LONG")]=macd_26
    candlesDataBase_BigQuery["MACD_TRIGGER"]=macd_9
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN SUPERTREND
def superTrend(df, atr_period, multiplier):
    
    high = df['HIGH']
    low = df['LOW']
    close = df['CLOSE']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    #return pd.DataFrame({
    #    'Supertrend': supertrend,
    #    'Final Lowerband': final_lowerband,
    #    'Final Upperband': final_upperband
    #}, index=df.index)

    df["SUPERTREND"]=supertrend

    return df

#_____

#CREATE VARIABLES
def variableCreation(candlesDataBase_BigQuery):
    
    global fractal
    updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")
    
    candlesDataBase_BigQuery=RSI(candlesDataBase_BigQuery)
#     candlesDataBase_BigQuery=MACD(candlesDataBase_BigQuery)
#     candlesDataBase_BigQuery=fibonacciLevels(candlesDataBase_BigQuery)
#     candlesDataBase_BigQuery=newVariables(candlesDataBase_BigQuery)
#     candlesDataBase_BigQuery=superTrend(candlesDataBase_BigQuery, int(get_config("PARAMETERS","ATR_PERIOD")), int(get_config("PARAMETERS","ATR_MULTIP")))
#     candlesDataBase_BigQuery=suaviPlusDerivates(candlesDataBase_BigQuery)
#     candlesDataBase_BigQuery=LAGS(candlesDataBase_BigQuery)
    
    candlesDataBase_BigQuery.dropna(inplace=True)
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE ESCALA LAS VARIABLES CONTINUAS DE LA BASE DE DATOS
def scaleVariables(candlesDataBase_BigQuery):
    
    #_____ESCALAR VARIABLES
    continuous_variables=list(candlesDataBase_BigQuery.columns)[3:-1]

    scaler=preprocessing.StandardScaler(with_mean=True).fit(np.array(candlesDataBase_BigQuery[continuous_variables]))
    scaled_data=pd.DataFrame(scaler.transform(candlesDataBase_BigQuery[continuous_variables]))
    scaled_data.columns=continuous_variables

    for var in continuous_variables:
        candlesDataBase_BigQuery[str(var)] = scaled_data[str(var)]
    
    #_____RETURN
    return candlesDataBase_BigQuery

#_____

#FUNCIÓN QUE CALCULA LÍMITE MÍNIMO DE EJECUCIÓN SEGÚN MINMAX SEMANAL, AYER, Y HOY
def portfolio_minmax(symbol):
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('config_surfNet_MonoBot.ini')
    
    #_____PARAMETERS
    t_frame = '1d'
    
    #_____CREATE DATAFRAME
    datos = ccxt.binance().fetch_ohlcv(symbol, t_frame)
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(datos, columns=header).set_index('Timestamp')
    df.index = df.index/1000
    df['Date'] = pd.to_datetime(df.index,unit='s')
    data =df.sort_values('Date',ascending=False)
    
    #_____CREATE TIME PARTITIONS
    semana = data.head(n=7)
    hoy = data.head(n=1)
    ayer= data.iloc[1]

    #_____CREATE INDICATORS (MEAN & LOW & CHANGE)
    mean_s = semana['High'].mean()
    mean_h = hoy['High'].mean()
    mean_a = ayer['High'].mean()
    Low_h = hoy['Low'].mean()
    Low_a = ayer['Low'].mean()
    change_a = mean_a/Low_a-1
    change_h = mean_h/Low_h-1
    
    #_____CREATE ALPHA BASED ON CHANGE
    change_variation_threshold = float(get_config('PORTFOLIO', 'CHANGE_VARIATION_THRESHOLD'))
    alpha_variation=float(get_config('PORTFOLIO', 'ALPHA_VARIATION'))
    if (change_a > change_variation_threshold) or (change_h>change_variation_threshold):
        por_debajo = 1-alpha_variation*float(get_config('PORTFOLIO', 'ALPHA_MULTIPLIER'))
    else:
        por_debajo = 1-alpha_variation
    
    #_____CREATE LIMIT BASED ON MEAN + ALPHA
    if mean_h < mean_a and mean_h < mean_s:
        limite = mean_h*por_debajo
    elif mean_a < mean_h and mean_a < mean_s:
        limite = mean_a*por_debajo
    else:
        limite = mean_s*por_debajo
        
    #_____HOW FAR IS PRICE FROM LIMIT
    closeValue=hoy["Close"].values[0]
    limite=limite/closeValue-1
        
    return limite

#_____

#FUNCIÓN QUE CALCULA LA RENTABILIDAD DE LAS HOLAS TOMANDO LAS VELAS DE X TIMEPO
def rentabilidad_promedio(symbol):
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('config_surfNet_MonoBot.ini')

    datos = ccxt.binance().fetch_ohlcv(symbol, get_config('PORTFOLIO', 'RETURN_EVALUATION_PERIOD'),limit=int(get_config('PORTFOLIO', 'RETURN_EVALUATION_PERIOD_LIMIT')))
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(datos, columns=header).set_index('Timestamp')
    df.index = df.index/1000
    df['Date'] = pd.to_datetime(df.index,unit='s')

    return np.mean(df.Close.pct_change().dropna().values)

#_____

#FUNCIÓN QUE CALCULA LA RENTABILIDAD PROMEDIO DE LAS VELAS TOMANDO EN X TIEMPO EN VELAS DE X
def amplitud_promedio(symbol):
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('config_surfNet_MonoBot.ini')
    
    #_____PARAMETERS
    amplitud = list()
    
    #_____CREATE DATABASE
    datos = ccxt.binance().fetch_ohlcv(symbol, get_config('PORTFOLIO', 'AMPLITUD_PERIOD_EVALUATION'))
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(datos, columns=header).set_index('Timestamp')
    df.index = df.index/1000
    df['Date'] = pd.to_datetime(df.index,unit='s')
    data =df.sort_values('Date',ascending=False)
    
    data.Close.pct_change()
    
    #_____CALCULATE AMPLITUDE
    mediahora = data.head(n=int(get_config('PORTFOLIO', 'AMPLITUD_PERIOD_EVALUATION_LAG')))
    for i in range(0,int(get_config('PORTFOLIO', 'AMPLITUD_PERIOD_EVALUATION_LAG'))):
        high = mediahora.iloc[i, 1]
        low = mediahora.iloc[i, 2]
        amplitud.append(high/low-1)
    amplitudP = np.mean(amplitud)
    
    return amplitudP

#_____

#FUNCIÓN QUE MIRA LA TENDENCIA DE LAS ÚLTIMAS X HORAS DE UN MERCADO
def marketTrend(market):
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('config_surfNet_MonoBot.ini')

    #_____CREATE DATABASE
    datos = ccxt.binance().fetch_ohlcv(market, get_config('PORTFOLIO', 'TREND_EVALUATION_PERIOD'),limit=int(get_config('PORTFOLIO', 'TREND_EVALUATION_PERIOD_LIMIT')))
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(datos, columns=header)
    df.Timestamp = df.Timestamp/1000
    df['Timestamp'] = pd.to_datetime(df.Timestamp,unit='s')
    df['Index']=df.index

    #_____CREATE TRENDLINE
    # fig, ax = plt.subplots()
    # df.plot(x='Index', y='Close', ax=ax)
    model = sm.formula.ols(formula='Close ~ Index', data=df[["Index","Close"]])
    res = model.fit()
    # df.assign(fit=res.fittedvalues).plot(x='Index', y='fit', ax=ax)
    marketTrend=res.fittedvalues.values[-1]/res.fittedvalues.values[0]-1
    
    return marketTrend

#_____

#FUNCIÓN QUE MIRA LA TENDENCIA DE LAS ÚLTIMAS X HORAS DE UN MERCADO
def volumenPromedio(market):
    
    #_____ACTUALIZAR ARCHIVO CONFIG
    updateConfig('config_surfNet_MonoBot.ini')

    #_____CREATE DATABASE
    datos = ccxt.binance().fetch_ohlcv(market, get_config('PORTFOLIO', 'VOLUME_EVALUATION_PERIOD'),limit=int(get_config('PORTFOLIO', 'VOLUME_EVALUATION_PERIOD_LIMIT')))
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(datos, columns=header)
    df.Timestamp = df.Timestamp/1000
    df['Timestamp'] = pd.to_datetime(df.Timestamp,unit='s')
    df['Index']=df.index
    df["Vol_Price"]=df["Volume"]*df["Close"]

    return np.mean(list(df.Vol_Price.values))

#_____

#FUNCIÓN QUE CREA PORTAFOLIO SEGÚN LÍNEA DE EFICIENCIA DE PARETO
def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs>costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient
    
#_____

#FUNCIÓN QUE DEVUELVE EL TOG X DE MERCADOS
def topCoinMarketCap():
    
    updateConfig("semiManual.ini")
    
    API_KEY_CAP="8edb78ea-b23d-4b23-8a6f-5ece6b703720"
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit': get_config("MARKETS","TOP_CMC"),
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': API_KEY_CAP,
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    response = pd.DataFrame(data["data"])
    
    return list(response.symbol.values)

# CHECKS

# RUN

In [ ]:
Pareto_df=pd.DataFrame(columns=["MARKET","TREND","AMPLITUD","RENTABILIDAD","LIMIT","VOLUME","RSI"])
writePickleVariable(Pareto_df,"portfolio")

#_____LOOP INFINITO
while True:
    
    #_____TRY DE CONTINGENCIA
    try:

        #_____ACTUALIZAR ARCHIVO CONFIG
        updateConfig('config_surfNet_MonoBot.ini')

        #_____ACTUALIZAR CREDENCIALES
        binance = ccxt.binance()
        binance.apiKey = get_config('BINANCE_PORTFOLIO', 'API_KEY')
        binance.secret = get_config('BINANCE_PORTFOLIO', 'API_SECRET')

        #_____NUEVA LISTA DE MERCADOS
        mercados=list(readPickleVariable("/home/ubuntu/SurfNet/predictionModels/PREDICTION/marketDict").keys())
            
        #_____UPDATE CONFIG PREDICTION
        updateConfig("/home/ubuntu/SurfNet/predictionModels/PREDICTION/predictionModels.ini")

        #_____CREAR PORTAFOLIO CON MERCADOS EXISTENTES
        mercados_Portafolio_Neto=[]
        for i in mercados:
            #_____ITERAR SOBRE FRACTAL
            for fractal in ast.literal_eval(get_config("PARAMETERS","FRACTAL_BUY")):
                try:
                    #_____DESCARGAR PARÁMETROS DE CORRECCIÓN
                    market_Parameters=readPickleVariable("/home/ubuntu/SurfNet/predictionModels/MODELS/"+i.replace("/","_")+"_PARAMETERS"+"_"+fractal)
                    if market_Parameters.at[0,"NET_UTILITY"]!=0:
                        mercados_Portafolio_Neto.append(i)
                except:
                    pass
            
        #_____ACTUALIZAR ARCHIVO CONFIG
        updateConfig('config_surfNet_MonoBot.ini')
            
        #_____REMOVER BNB POR TEMA DE PAGO DE COMISIONES
        try:
            mercados_Portafolio_Neto.remove("BNB/USDT")
        except:
            pass
            
        #_____SI LISTA DE MERCADOS CONTIENE VALORES
        if len(mercados_Portafolio_Neto) >= 1:
            
            #_____NUEVA LISTA DE MERCADOS
            topMarketList=[]
            
            #_____CRUZAR MERCADOS CON TOP EN COINMARKETCAP
            for market in topCoinMarketCap():
                if market+"/USDT" in mercados_Portafolio_Neto:
                    topMarketList.append(market+"/USDT")
                
            #_____ACTUALIZAR ARCHIVO CONFIG
            updateConfig('config_surfNet_MonoBot.ini')

            #_____PORTAFOLIO GENERAL
            marketPortfolioDataFrame=pd.DataFrame()
            marketPortfolioDataFrame_contador=0
            
            for i in topMarketList:
                marketPortfolioDataFrame.at[marketPortfolioDataFrame_contador,"MARKET"]=i
                
                if int(get_config("PORTFOLIO","TREND"))==1:
                    marketPortfolioDataFrame.at[marketPortfolioDataFrame_contador,"TREND"]=marketTrend(i)
                    time.sleep(5)
                if int(get_config("PORTFOLIO","AMPLITUD"))==1:
                    marketPortfolioDataFrame.at[marketPortfolioDataFrame_contador,"AMPLITUD"]=amplitud_promedio(i)
                    time.sleep(5)
                if int(get_config("PORTFOLIO","RENTABILIDAD"))==1:
                    marketPortfolioDataFrame.at[marketPortfolioDataFrame_contador,"RENTABILIDAD"]=rentabilidad_promedio(i)
                    time.sleep(5)
                if int(get_config("PORTFOLIO","LIMIT"))==1:
                    marketPortfolioDataFrame.at[marketPortfolioDataFrame_contador,"LIMIT"]=portfolio_minmax(i)
                    time.sleep(5)
                if int(get_config("PORTFOLIO","VOLUME"))==1:
                    marketPortfolioDataFrame.at[marketPortfolioDataFrame_contador,"VOLUME"]=volumenPromedio(i)
                    time.sleep(5)
                
                marketPortfolioDataFrame_contador=marketPortfolioDataFrame_contador+1

            #_____RELEVANT VARIABLES
            contador=0
            for market_model in list(marketPortfolioDataFrame.MARKET.values):

                #_____CARGAR BASE DE DATOS
                myData = binance.fetch_ohlcv(market_model,"15"+"m",limit=1000)
                time.sleep(5)
                candlesDataBase_BigQuery =pd.DataFrame(columns=["ID","TIME","MARKET","OPEN","HIGH","LOW","CLOSE","VOLUME"])

                for i in range(0,len(myData)):
                    candlesDataBase_BigQuery.at[i,"ID"]=myData[i][0]
                    candlesDataBase_BigQuery.at[i,"TIME"]=datetime.fromtimestamp(myData[i][0]/1000.0)
                    candlesDataBase_BigQuery.at[i,"MARKET"]=market_model
                    candlesDataBase_BigQuery.at[i,"OPEN"]=myData[i][1]
                    candlesDataBase_BigQuery.at[i,"HIGH"]=myData[i][2]
                    candlesDataBase_BigQuery.at[i,"LOW"]=myData[i][3]
                    candlesDataBase_BigQuery.at[i,"CLOSE"]=myData[i][4]
                    candlesDataBase_BigQuery.at[i,"VOLUME"]=myData[i][5]

                #_____GUARDAR BASE DE DATOS PARA BACKTESTING FUTURO
                backtestingDataFrame=candlesDataBase_BigQuery.copy()

                #_____CREAR VARIABLES INPUT
                candlesDataBase_BigQuery=variableCreation(candlesDataBase_BigQuery)

                #####

                marketPortfolioDataFrame.at[contador,"RSI"]=(candlesDataBase_BigQuery.at[len(candlesDataBase_BigQuery)-1,"RSI"])*(-1)
                
                #####

                contador+=1
                

            #_____PORTAFOLIO JERÁRQUICO
            Pareto_df=marketPortfolioDataFrame.copy()
            Pareto_df.reset_index(inplace=True,drop=True)
            Pareto_df_numpy=Pareto_df.drop(['MARKET'], axis=1)

            Efficient_market=is_pareto_efficient(np.array(Pareto_df_numpy), return_mask = False)
            Pareto_df=Pareto_df.loc[list(Efficient_market),:]

            writePickleVariable(Pareto_df,"portfolio")
            
        else:
            
            Pareto_df=pd.DataFrame(columns=["MARKET","TREND","AMPLITUD","RENTABILIDAD","LIMIT","VOLUME"])
            writePickleVariable(Pareto_df,"portfolio")
        
        #_____CLEAR
        clear_output()
        
        #_____SLEEP
        time.sleep(300)
        
    #_____EXCEPT
    except:
        
        #_____PRINT
        print("[[ERROR]]")
        
        #_____SLEEP
        time.sleep(300)